In [294]:
import pandas as pd
import numpy as np
import datetime

#### read file and preprocess some raw data

In [303]:
p = pd.read_csv("patient.csv")
weather = pd.read_csv("weather.csv")
phu = pd.read_csv("phu.csv")
special = pd.read_csv("special.csv")
date = pd.read_csv("date.csv")
mobility = pd.read_csv("mobility.csv")
covid = pd.read_csv("./covid/covid2.csv")
fact =  pd.DataFrame(columns=['date_key','mobility_key','patient_key','special_measures_key','weather_key','phu_location_key','fatal','unresolved','resolved'])

In [304]:
special['start_date'] = pd.to_datetime(special['start_date'])
special['end_date'] = pd.to_datetime(special['end_date'])
covid['Accurate_Episode_Date'] = pd.to_datetime(covid['Accurate_Episode_Date'])
covid['Test_Reported_Date'] = pd.to_datetime(covid['Test_Reported_Date'])
date['date'] = pd.to_datetime(date['date'])

covid = covid[(covid['Accurate_Episode_Date'] >= '10/16/2020') & (covid['Accurate_Episode_Date'] <= '02/16/2021')]
covid = covid[(covid['Test_Reported_Date'] >= '10/16/2020') & (covid['Test_Reported_Date'] <= '02/16/2021')]
covid = covid[(covid['Reporting_PHU_City'] == 'Toronto') | (covid['Reporting_PHU_City'] == 'Mississauga') 
              |(covid['Reporting_PHU_City'] == 'Ottawa') |(covid['Reporting_PHU_City'] == 'Oakville')
              |(covid['Reporting_PHU_City'] == 'Whitby')|(covid['Reporting_PHU_City'] == 'Newmarket')]

In [305]:
covid

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1792,2020-10-16,2020-10-21,2020-10-21,2020-10-19,50s,MALE,TRAVEL,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
1,1808,2020-10-16,2020-10-23,2020-10-23,2020-10-21,70s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
2,1810,2020-10-16,2020-10-23,2020-10-23,2020-10-21,60s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
3,1823,2020-10-16,2020-10-21,2020-10-21,2020-10-19,20s,MALE,NO KNOWN EPI LINK,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
4,1826,2020-10-16,2020-10-21,2020-10-21,2020-10-19,40s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162223,301847,2021-02-16,2021-02-19,2021-02-19,2021-02-18,40s,FEMALE,NO KNOWN EPI LINK,Resolved,n,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
162224,303576,2021-02-16,2021-03-03,2021-03-03,2021-03-01,40s,FEMALE,NO KNOWN EPI LINK,Resolved,n,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
162225,304577,2021-02-16,2021-03-01,2021-03-01,2021-02-25,<20,MALE,CC,Resolved,n,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
162226,304945,2021-02-16,2021-03-02,2021-03-02,2021-03-01,60s,MALE,CC,Resolved,n,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239


In [306]:
covid.reset_index(drop=True,inplace=True)
covid['Row_ID'] = covid.index+1
covid

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,2020-10-16,2020-10-21,2020-10-21,2020-10-19,50s,MALE,TRAVEL,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
1,2,2020-10-16,2020-10-23,2020-10-23,2020-10-21,70s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
2,3,2020-10-16,2020-10-23,2020-10-23,2020-10-21,60s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
3,4,2020-10-16,2020-10-21,2020-10-21,2020-10-19,20s,MALE,NO KNOWN EPI LINK,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
4,5,2020-10-16,2020-10-21,2020-10-21,2020-10-19,40s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162223,162224,2021-02-16,2021-02-19,2021-02-19,2021-02-18,40s,FEMALE,NO KNOWN EPI LINK,Resolved,n,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
162224,162225,2021-02-16,2021-03-03,2021-03-03,2021-03-01,40s,FEMALE,NO KNOWN EPI LINK,Resolved,n,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
162225,162226,2021-02-16,2021-03-01,2021-03-01,2021-02-25,<20,MALE,CC,Resolved,n,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
162226,162227,2021-02-16,2021-03-02,2021-03-02,2021-03-01,60s,MALE,CC,Resolved,n,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239


#### get total fatal, unresolved, resolved numbers

In [307]:
print(sum(covid['Outcome1'] == 'Fatal'))
print(sum(covid['Outcome1'] == 'Not Resolved'))
print(sum(covid['Outcome1'] == 'Resolved'))

2225
77
159926


#### assign patient_key

In [308]:
fact['patient_key'] = covid['Row_ID']
fact

,date_key,mobility_key,patient_key,special_measures_key,weather_key,phu_location_key,fatal,unresolved,resolved
0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
162223,NaN,NaN,162224,NaN,NaN,NaN,NaN,NaN,NaN
162224,NaN,NaN,162225,NaN,NaN,NaN,NaN,NaN,NaN
162225,NaN,NaN,162226,NaN,NaN,NaN,NaN,NaN,NaN
162226,NaN,NaN,162227,NaN,NaN,NaN,NaN,NaN,NaN


#### set phu_location_key

In [309]:
def set_phu(x):
    if x == 'Ottawa Public Health':
        return 1
    elif x == 'Toronto Public Health':
        return 2
    elif x == 'Peel Public Health':
        return 3
    elif x == 'Durham Region Health Department':
        return 4
    elif x == 'Halton Region Health Department':
        return 5
    else:
        return 6

In [310]:
fact['phu_location_key'] = covid['Reporting_PHU'].apply(set_phu)

In [311]:
fact

,date_key,mobility_key,patient_key,special_measures_key,weather_key,phu_location_key,fatal,unresolved,resolved
0,NaN,NaN,1,NaN,NaN,1,NaN,NaN,NaN
1,NaN,NaN,2,NaN,NaN,1,NaN,NaN,NaN
2,NaN,NaN,3,NaN,NaN,1,NaN,NaN,NaN
3,NaN,NaN,4,NaN,NaN,1,NaN,NaN,NaN
4,NaN,NaN,5,NaN,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
162223,NaN,NaN,162224,NaN,NaN,2,NaN,NaN,NaN
162224,NaN,NaN,162225,NaN,NaN,6,NaN,NaN,NaN
162225,NaN,NaN,162226,NaN,NaN,2,NaN,NaN,NaN
162226,NaN,NaN,162227,NaN,NaN,6,NaN,NaN,NaN


#### set date_key

In [312]:
date

,date_key,date,day,month,day_num_of_week,week_num_of_year,weekend,holiday,season
0,20201016,2020-10-16,16,10,5,42,y,n,Autumn
1,20201017,2020-10-17,17,10,6,42,y,n,Autumn
2,20201018,2020-10-18,18,10,7,42,y,n,Autumn
3,20201019,2020-10-19,19,10,1,43,n,n,Autumn
4,20201020,2020-10-20,20,10,2,43,n,n,Autumn
...,...,...,...,...,...,...,...,...,...
119,20210212,2021-02-12,12,2,5,6,y,n,Winter
120,20210213,2021-02-13,13,2,6,6,y,n,Winter
121,20210214,2021-02-14,14,2,7,6,y,n,Winter
122,20210215,2021-02-15,15,2,1,7,n,y,Winter


In [313]:
covid

,Row_ID,Accurate_Episode_Date,Case_Reported_Date,Test_Reported_Date,Specimen_Date,Age_Group,Client_Gender,Case_AcquisitionInfo,Outcome1,Outbreak_Related,Reporting_PHU_ID,Reporting_PHU,Reporting_PHU_Address,Reporting_PHU_City,Reporting_PHU_Postal_Code,Reporting_PHU_Website,Reporting_PHU_Latitude,Reporting_PHU_Longitude
0,1,2020-10-16,2020-10-21,2020-10-21,2020-10-19,50s,MALE,TRAVEL,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
1,2,2020-10-16,2020-10-23,2020-10-23,2020-10-21,70s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
2,3,2020-10-16,2020-10-23,2020-10-23,2020-10-21,60s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
3,4,2020-10-16,2020-10-21,2020-10-21,2020-10-19,20s,MALE,NO KNOWN EPI LINK,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
4,5,2020-10-16,2020-10-21,2020-10-21,2020-10-19,40s,FEMALE,CC,Resolved,n,2251,Ottawa Public Health,100 Constellation Drive,Ottawa,K2G 6J8,www.ottawapublichealth.ca,45.345665,-75.763912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162223,162224,2021-02-16,2021-02-19,2021-02-19,2021-02-18,40s,FEMALE,NO KNOWN EPI LINK,Resolved,n,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
162224,162225,2021-02-16,2021-03-03,2021-03-03,2021-03-01,40s,FEMALE,NO KNOWN EPI LINK,Resolved,n,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
162225,162226,2021-02-16,2021-03-01,2021-03-01,2021-02-25,<20,MALE,CC,Resolved,n,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
162226,162227,2021-02-16,2021-03-02,2021-03-02,2021-03-01,60s,MALE,CC,Resolved,n,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239


In [314]:
def set_special(x):
    if x == datetime.datetime(2020,12,26):
        return 8
    elif x == datetime.datetime(2020,12,15):
        return 6
    elif x == datetime.datetime(2020,12,27):
        return 9
    elif (x >= datetime.datetime(2020,10,16)) & (x <= datetime.datetime(2020,11,7)):
        return 1
    elif (x >= datetime.datetime(2020,11,7)) & (x < datetime.datetime(2020,11,14)):
        return 2
    elif (x >= datetime.datetime(2020,11,14)) & (x < datetime.datetime(2020,12,7)):
        return 3
    elif (x >= datetime.datetime(2020,12,7)) & (x < datetime.datetime(2020,12,14)):
        return 4
    elif (x >= datetime.datetime(2020,12,14)) & (x < datetime.datetime(2020,12,16)):
        return 5
    elif (x >= datetime.datetime(2020,12,16)) & (x < datetime.datetime(2021,1,14)):
        return 7
    elif (x >= datetime.datetime(2021,1,14)) & (x < datetime.datetime(2021,2,14)):
        return 10
    return 11

In [315]:
fact['special_measures_key'] = covid['Test_Reported_Date'].apply(set_special)
fact

,date_key,mobility_key,patient_key,special_measures_key,weather_key,phu_location_key,fatal,unresolved,resolved
0,NaN,NaN,1,1,NaN,1,NaN,NaN,NaN
1,NaN,NaN,2,1,NaN,1,NaN,NaN,NaN
2,NaN,NaN,3,1,NaN,1,NaN,NaN,NaN
3,NaN,NaN,4,1,NaN,1,NaN,NaN,NaN
4,NaN,NaN,5,1,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
162223,NaN,NaN,162224,11,NaN,2,NaN,NaN,NaN
162224,NaN,NaN,162225,11,NaN,6,NaN,NaN,NaN
162225,NaN,NaN,162226,11,NaN,2,NaN,NaN,NaN
162226,NaN,NaN,162227,11,NaN,6,NaN,NaN,NaN


In [316]:
def set_mobility(x):
    if x == 'Ottawa Public Health':
        return 1
    elif x == 'Toronto Public Health':
        return 2
    elif x == 'Peel Public Health':
        return 3
    elif x == 'Durham Region Health Department':
        return 4
    elif x == 'Halton Region Health Department':
        return 5
    else:
        return 6

In [317]:
fact['mobility_key'] = covid['Reporting_PHU'].apply(set_mobility)
fact

,date_key,mobility_key,patient_key,special_measures_key,weather_key,phu_location_key,fatal,unresolved,resolved
0,NaN,1,1,1,NaN,1,NaN,NaN,NaN
1,NaN,1,2,1,NaN,1,NaN,NaN,NaN
2,NaN,1,3,1,NaN,1,NaN,NaN,NaN
3,NaN,1,4,1,NaN,1,NaN,NaN,NaN
4,NaN,1,5,1,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
162223,NaN,2,162224,11,NaN,2,NaN,NaN,NaN
162224,NaN,6,162225,11,NaN,6,NaN,NaN,NaN
162225,NaN,2,162226,11,NaN,2,NaN,NaN,NaN
162226,NaN,6,162227,11,NaN,6,NaN,NaN,NaN


In [320]:
def outcome_fatal(x):
    if (x == "Fatal"):
        return 1

fact['fatal'] = covid['Outcome1'].apply(outcome_fatal)

def outcome_unresolved(x):
    if (x == "Not Resolved"):
        return 1

fact['unresolved'] = covid['Outcome1'].apply(outcome_unresolved)

def outcome_resolved(x):
    if (x == "Resolved"):
        return 1
fact['resolved'] = covid['Outcome1'].apply(outcome_resolved)
    
fact['fatal'] = fact['fatal'].fillna(0)
fact['unresolved'] = fact['unresolved'].fillna(0)
fact['resolved'] = fact['resolved'].fillna(0)

In [321]:
fact

,date_key,mobility_key,patient_key,special_measures_key,weather_key,phu_location_key,fatal,unresolved,resolved
0,NaN,1,1,1,NaN,1,0.0,0.0,1.0
1,NaN,1,2,1,NaN,1,0.0,0.0,1.0
2,NaN,1,3,1,NaN,1,0.0,0.0,1.0
3,NaN,1,4,1,NaN,1,0.0,0.0,1.0
4,NaN,1,5,1,NaN,1,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
162223,NaN,2,162224,11,NaN,2,0.0,0.0,1.0
162224,NaN,6,162225,11,NaN,6,0.0,0.0,1.0
162225,NaN,2,162226,11,NaN,2,0.0,0.0,1.0
162226,NaN,6,162227,11,NaN,6,0.0,0.0,1.0


In [322]:
fact.to_csv("fact.csv",index=False)